In [270]:
import numpy as np
import pandas as pd

# Exercise 1 (Bruteforce T=2)

In [207]:
# struct like object
class struct():
    pass

def get_id_of_maxval(array):
    """array : np.array object"""
    return np.argmax(array)

In [161]:
# Setup
param = struct()
param.alpha = 0.5
param.beta = 0.9
param.t = 2
param.u = lambda x, par: x**par.alpha 

In [162]:
M = 5

In [165]:
# setup
Vstar = - np.inf
Cstar = [np.nan for _ in range(param.t)]

for C1 in range(0, M + 1):
    # evaluate 
    V = param.u(C1, param) + param.beta * param.u(M-C1, param) 
    if V > Vstar:
        Vstar = V
        Cstar[0] = C1
        Cstar[1] = M - C1

print('Vstar', Vstar)
print('Cstar', Cstar)


Vstar 3.004843013704663
Cstar [3, 2]


# Exercise 2 (Bruteforce T=3)

In [245]:
# Setup
param = struct()
param.alpha = 0.5
param.beta = 0.9
param.t = 3
param.u = lambda x, par: x**par.alpha 

In [246]:
M = 5

In [247]:
# setup
Vstar = - np.inf
Cstar = [np.nan for _ in range(param.t)]

In [248]:
for C2 in range(0, M+1):
    for C1 in range(0, M-C2): #here note that it has a variable length for end point
        
        V = param.u(C2,param) + param.beta * param.u(C1,param) + (param.beta**2)*param.u(M-C1-C2,param)
        
        if V > Vstar:
            Vstar = V
            Cstar[0] = C2
            Cstar[1] = C1
            Cstar[2] = M - C1 - C2
            
print('Vstar',Vstar)
print('Cstar',Cstar)
        

Vstar 3.4970057685088807
Cstar [2, 2, 1]


# Backwards induction (T=3)

In [263]:
# Setup
param = struct()
param.alpha = 0.5
param.beta = 0.9
param.T = 10
param.u = lambda x, par: x**par.alpha 

In [264]:
M = 5

In [265]:
def find_V(M, Vstar_t, Vstar_next, Cstar_t, param):
    
    for m in range(0, M+1):
        Vstar_t[m] = -np.inf

        for Ct in range(m+1):
            i_next = m - Ct
            V = np.sqrt(Ct) + param.beta * Vstar_next[i_next]
            if V > Vstar_t[m]:
                Vstar_t[m] = V
                Cstar_t[m] = Ct
    return Vstar_t, Cstar_t


In [266]:
Vstar, Cstar = dict(), dict()

Vstar[param.T - 1] = param.u(np.arange(M+1), param)
Cstar[param.T - 1] = np.arange(M+1)

for t in reversed(range(param.T - 1)):
    
    #print('this t', t)
    Vstar[t] = np.ones(M+1)*(np.nan)
    Cstar[t] = np.ones(M+1)*(np.nan)
    
    Vstar[t], Cstar[t] = find_V(M, Vstar[t], Vstar[t+1] , Cstar[t], param)
    


In [267]:
def get_ct(Vstar_t, M, C_acc):
    consumption_limit = int(M - C_acc)
    idx = np.argmax(Vstar_t[0: consumption_limit + 1]) #remember index error needs plus 1
    return idx

In [268]:
consumption_path = dict()
C_acc = 0
for t in sorted(Cstar.keys()):
    i = get_ct(Vstar[t], M, C_acc) #np.argmax get's id of highest value
    Ct = Cstar[t][i]
    C_acc = C_acc + Ct
    consumption_path[t] = Ct


In [285]:
res = pd.DataFrame(zip(*consumption_path.items())).T
res.columns = ['time', 'consumption']
res.set_index('time')

,consumption
time,
0.0,1.0
1.0,1.0
2.0,1.0
3.0,1.0
4.0,1.0
5.0,0.0
6.0,0.0
7.0,0.0
8.0,0.0
